# ML Deep Dive: Probability and Naive Bayes

In this tutorial I'll do a deep dive of Naive Bayes, a classical classification algorithm. Naive Bayes is very simple to understand (once you understand Bayes Rule), and has historically worked well on many classification problems, especially text classification. Naive Bayes is perhaps the most important supervised learning algorithm in NLP, or at least was before deep learning became competitive.

Perhaps more than other ML algorithms, understanding Naive Bayes requires some understanding of basic probability theory. Since probability theory isn't a prerequisite for these tutorials, I'll now give a brief intro to probability theory with the main goal of understanding Bayes Rule, which is the core formula used in Naive Bayes. If you already have a basic understanding of probability feel free to skip the first part of this tutorial.

In [153]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter, defaultdict

# Sklearn Packages (mainly for utility functions)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import *

from utils import get_spam_data

np.random.seed(42)

## Introduction to Probability

### Motivation

Probability theory is essentially a calculus for modeling random quantities, a way of taking into account the fact that real world data is usually noisy. For practical purposes, you can think of randomness as something you can't predict because you don't have enough information. If you could understand and model your problem perfectly, taking all variables into account with perfect precision, then you wouldn't need to model randomness because there wouldn't be any. Thus, randomness is an expression of ignorance (aka *entropy*). The more you know and can account for, the less random your data is. Since it's usually impossible to know everything about your problem, randomness is something you have to deal with in real life, which is what probability (and statistics) is for.

### Basics
In probability lingo, a random quantity is called a *random variable*. A random variable can be either a number or a collection of numbers (you can have random vectors, random matrices, random tensors, random functions, etc). Similar to how you can think of a data point as some variable taking on a value, you can think of a (random) data point as some random variable taking on a value. The difference with random variables is you can't guarantee it'll take on the value you want it to. You can imagine some process you can't see adding some noise to the value you want, and giving you that noisy value instead. Instead of getting `x = 12`, you might get `x = 12 + error`, where `error` is something you can't predict exactly.

We can see an example of how this randomness works by using the `np.random.rand` function, which generates a (pseudo) random number between 0 and 1. (Aside: For an in depth discussion of how to generate random numbers, see my tutorial on that.) I use the rand function below to create an `error` a few times so you can see how randomness can make exact values inexact. Notice you never get `x=12` exactly, which would be very unlikely.

In [2]:
x = 12
print(f'exact value: {x}')
for _ in range(10):
    error = np.random.rand()
    print(f'random value: {x + error}')

exact value: 12
random value: 12.374540118847362
random value: 12.950714306409916
random value: 12.731993941811405
random value: 12.598658484197037
random value: 12.156018640442436
random value: 12.155994520336202
random value: 12.058083612168199
random value: 12.866176145774935
random value: 12.60111501174321
random value: 12.708072577796045


Rather than random variables taking on a *value*, they take on a *distribution* of values. A distribution is a set of probabilities assigned to the values that random variable can take on. Each such value has a probability associated with it. Intuitively, you can think of the probability of a value as its weight, its likelihood of being sampled. Values with higher weights have higher probabilities. Values with lower weights have lower probabilities.

In math terms, a *probability* defined over a collection of values $x$ is a function $p(x)$ whose values are always positive and sum to one. More formally, suppose $x$ is a random variable that can take on $n$ values $x_1, x_2, \cdots, x_n$. Then

$$p(x) \geq 0 \quad \text{for all } x=x_1, x_2, \cdots, x_n,$$
$$\sum_i^n p(x_i) = p(x_1) + p(x_2) + \cdots + p(x_n) = 1.$$

We say $p(x)$ is the "probability that $x$ will occur". One can *define* then the distribution of values $x_1, x_2, \cdots, x_n$ by the function $p(x)$ itself, or equivalently by the set of probability values $p(x_1), p(x_2), \cdots, p(x_n)$.

Notice by this definition, any set of positive weights $w_1, w_2, \cdots, w_n$ attached to $x_1, x_2, \cdots, x_n$ can be made into probabilities by simply dividing those weights by their sum over all values:

$$p(x_i) = \frac{w_i}{\sum_j w_j}.$$

We can see an example of how this works below. I define an array of values `x` and an array of weights `w` for those values, and use those to calculate an array of probabilities `p`. Observe that the probabilities created satisfy the definition above, namely each element of `p` is positive, and `sum(p)=1`. Note for this to work it's clearly important that `x` and `w` have the same size, and that each value of `w` is positive.

In [12]:
x = [-1, 3, -7, 4, 1, 0]
w = [1, 5, 2, 1, 3, 4]
p = [weight / sum(w) for weight in w]
print('x \t w \t p')
print('--- \t --- \t ---')
for a,b,c in zip(x, w, p):
    print(f'{a} \t {b} \t {c}')
print(f'sum of probs: {sum(p)}')

x 	 w 	 p
--- 	 --- 	 ---
-1 	 1 	 0.0625
3 	 5 	 0.3125
-7 	 2 	 0.125
4 	 1 	 0.0625
1 	 3 	 0.1875
0 	 4 	 0.25
sum of probs: 1.0


### Probabilities as Frequencies

There's another way to interpret probabilities that will be useful to us as well in the derivation of Naive Bayes, namely that of a frequency. Suppose you have a random sample of data, and each of your data points $x$ can take on any of the values $x_1,x_2,\cdots,x_k$ in some set. Let $n_j$ be the number of times the value $x_j$ occurs in your dataset. Then we can (approximately) define the probability $p(x=x_j)$ by

$$p(x=x_j) \approx \frac{n_j}{n} = \frac{\text{# times } x_j \text{ occurs in the dataset}}{\text{# of samples in the dataset}}.$$

This says that practically speaking, you can think of a probability of some value occuring as the ratio of the number of times it did happen to the number of times it could have happened in your data. Note in practice, you rarely know the *true* distribution your data was sampled from, and so you often either have to approximate it using this formula, or make some assumptions about how your data was distributed (i.e. define a model for your data). 

Also, note it must be true that $p(x_1)+\cdots+p(x_k)=1$. You can see from the above frequency formula that this follows automatically, as the number of times all possible values can occur is $n$, so $\frac{n_1+\cdots+n_k}{n}=\frac{n}{n}=1$.

I walk through an example of how to do this with some data below. In order to get probabilities this way it's useful to apply a _counter_ to the data. A counter is basically a dictionary where each key is a single value in the data, and each value is a count of how many times that value occurs in the data. They're fairly simple to implement from scratch if you're curious, and are often involved in common coding questions for software engineering interviews. Counters come pre-installed with python via the `collections` module, which is part of the python standard library.

We can easily use a counter to create probabilities on the values from the data by dividing the counts by the number of samples for each value (which is what the above formula says to do). Again, it should be clear that `sum(p)=1`.

In [22]:
data = [3, 4, 4, 5, 2, 4, 1, 9, 0, 10, 0, 9, 6, 1, 3, 4, 6, 1, 5, 0]
counter = Counter(data)
counter

Counter({3: 2, 4: 4, 5: 2, 2: 1, 1: 3, 9: 2, 0: 3, 10: 1, 6: 2})

In [35]:
n = len(data)
p = {x: counter[x] / n for x in counter}

print(f'x \t count \t p(x)')
print('--- \t --- \t ---')
for i in range(len(p)):
    print(f'{data[i]} \t {counter[data[i]]} \t {p[data[i]]}')
print(f'sum of probs: {sum(p.values())}')

x 	 count 	 p(x)
--- 	 --- 	 ---
3 	 2 	 0.1
4 	 4 	 0.2
4 	 4 	 0.2
5 	 2 	 0.1
2 	 1 	 0.05
4 	 4 	 0.2
1 	 3 	 0.15
9 	 2 	 0.1
0 	 3 	 0.15
sum of probs: 1.0


### Joint Distributions and Conditional Distributions

Just like we can talk about functions of multiple variables, like $z=f(x,y)$, we can talk about probabilities of multiple variables as well, like $p(x,y)$ (often called the *joint distribution* of $x$ and $y$). The two most important concepts to take away from joint probabilities are the concepts of *independence* and *conditional probability*. 

Informally, two random variables are independent when knowing the distribution of one tells you nothing about the distribution of the other; you can essentially think of them as two completely separate (i.e. independent) processes. In math terms, this just means the joint distribution factors, i.e. $p(x,y)=p(x)p(y)$. (Subtle point: Each $p$ has a different meaning in this formula. Writing $p(x,y)$ means "the distribution from which $x,y$ are jointly sampled in pairs". Writing $p(x)$ means "the distribution from which $x$ alone is sampled if we do so ignoring $y$". And writing $p(y)$ means "the distribution from which $y$ alone is sampled if we do so ignoring $x$". This abuse of notation is done all the time in probability. You can usually tell which $p$ is which by looking at what random variable it's a function of.)

Informally, conditionally probability is a measure of how non-independent (i.e. dependent) two random variables are. If $x$ and $y$ are dependent, knowing what $x$ is before sampling $y$ can completely affect what the distribution of $y$ ends up being. (For ordinary, non-random variables, this just says that $y$ would be a function of $x$, $y=y(x)$. Dependence is just an extension of that concept to random variables.). We measure the dependence of $y$ on $x$ by defining the conditional probability distribution $p(y|x)$, which reads as "the probability of sampling a value $y$ given we know the value of $x$ already", or in short "the probability of $y$ given $x$". It's defined mathematically by

$$p(y|x) = \frac{p(x,y)}{p(x)}.$$

Notice $p(y|x)=p(y)$ only when $x$ and $y$ are independent. Otherwise, knowing something about $x$ changes what distribution $y$ could be sampled from, as $p(y|x)$ is its own distribution completely different from $p(y)$. We often express the sampling of $y$ given $x$ as a "new" random variable $y|x$. 

It should hopefully be clear that we can just as easily define a distribution $p(x|y)$ as well, the probability of $x$ given $y$. Just swap $x$ and $y$ in the above. Note that, just as knowing something about $x$ can affect what $y$ is, knowing something about $y$ can affect what $x$ is. However, and this is very important, note in general $p(x|y) \neq p(y|x)$! You wouldn't believe how many people make this mistake when applying it to real life problems, even smart people.

Below is an example of how to work with joint probabilities. I create a small array `XY`, where the first column is the samples for `X` and the second column is the corresponding samples for `Y`. I calculate each of the distributions below. Can you verify that $p(y|x)=p(x,y)p(x)=p(x,y)p(y)$ in each of these examples? Can you verify that the sum of probabilities in each column is one (over unique `x,y` pairs), which must be true?

In [107]:
XY = np.array([[1, 1], [2, 0], [2, 0], [2, 1], [1, 1], [2, 0]])
X, Y = (XY[:, 0], XY[:, 1])

XY_tuples = [tuple(row) for row in XY]
xy_counter = Counter(XY_tuples)
pxy = {xy: xy_counter[xy] / len(XY) for xy in set(XY_tuples)}

x_counter = Counter(X)
px = {x: x_counter[x] / len(X) for x in set(X)}

y_counter = Counter(Y)
py = {y: y_counter[y] / len(Y) for y in set(Y)}

x_y1_counter = Counter(X[Y == 1])
px_y1 = {x: x_y1_counter[x] / len(X[Y == 1]) for x in set(X)}

x_y0_counter = Counter(X[Y == 0])
px_y0 = {x: x_y0_counter[x] / len(X[Y == 0]) for x in set(X)}

y_x1_counter = Counter(Y[X == 1])
py_x1 = {y: y_x1_counter[y] / len(Y[X == 1]) for y in set(Y)}

y_x2_counter = Counter(Y[X == 2])
py_x2 = {y: y_x2_counter[y] / len(Y[X == 2]) for y in set(Y)}

print('x \t y      p(x,y)   p(x)    p(y)  p(x|y=1)  p(x|y=0)  p(y|x=1)  p(y|x=2)')
print('-'*80)
for row in XY:
    x, y = tuple(row)
    a,b,c,d,e,f,g = (round(v, 2) for v in (pxy[(x,y)], px[x], py[y], px_y1[x], px_y0[x], py_x1[y], py_x2[y]))
    if x == 1:
        g = '-'
    if x == 2:
        f = '-'
    if y == 0:
        d = '-'
    if y == 1:
        e = '-'
    print(f'{x} \t {y} \t {a} \t {b} \t {c} \t {d} \t {e} \t   {f} \t     {g}')

x 	 y      p(x,y)   p(x)    p(y)  p(x|y=1)  p(x|y=0)  p(y|x=1)  p(y|x=2)
--------------------------------------------------------------------------------
1 	 1 	 0.33 	 0.33 	 0.5 	 0.67 	 - 	   1.0 	     -
2 	 0 	 0.5 	 0.67 	 0.5 	 - 	 1.0 	   - 	     0.75
2 	 0 	 0.5 	 0.67 	 0.5 	 - 	 1.0 	   - 	     0.75
2 	 1 	 0.17 	 0.67 	 0.5 	 0.33 	 - 	   - 	     0.25
1 	 1 	 0.33 	 0.33 	 0.5 	 0.67 	 - 	   1.0 	     -
2 	 0 	 0.5 	 0.67 	 0.5 	 - 	 1.0 	   - 	     0.75


### Bayes Rule

Conditional probability is what allows us to define the important formula for Naive Bayes: Bayes Rule. Bayes Rule is a way of relating the two conditional distributions $p(y|x)$ and $p(x|y)$. I just said these two are in general not equal to each other. How do we relate them then? The trick is to use the fact that 

$$p(x,y)=p(y|x)p(x)=p(x|y)p(y)$$

from the definition of conditional probability above. Solving the right two equations for $p(y|x)$ we arrive at

$$p(y|x) = \frac{p(x|y)p(y)}{p(x)}. \quad \quad \text{(Bayes Rule)}$$

Note in Bayesian lingo $p(y)$ in this formula is often called the *prior* distribution and $p(y|x)$ the *posterior* distribution. This comes from thinking of Bayes Rule as an update rule. Imagine you want to know something about $y$. You start with a "prior belief" about what $y$ might be, a guess. You then collect some measurements $x$, and would like to figure out how knowing something about $x$ changes your belief about what $y$ is, i.e. what $y|x$ is. You can thus think of the posterior $p(y|x)$ as your updated belief about $y$ given that you've seen $x$. The distribution $p(x|y)$ is often called the *likelihood*, which is the distribution you assume $x$ is being sampled from (i.e. it's your model of $x$ that you're trying to fit to your input data).

Using the above example `XY`, we can verify that Bayes Rule works by comparing the conditional probabilities $p(y|x)$ with $\frac{p(x|y)p(y)}{p(x)}$ and making sure they give the same answer. Note that $p(y|x=1)$ and $p(y|x=2)$ are two different distributions, hence $p(y=0|x=1) + p(y=1|x=1) = 1$ *and* $p(y=0|x=2) + p(y=1|x=2) = 1$.

In [106]:
print('p(y=0|x=1) = ( p(x=1|y=0) * p(y=0) ) / p(x=1)')
print(f'{py_x1[0]} = ( {px_y0[1]} * {py[0]} ) / {px[1]}')
print()

print('p(y=1|x=1) = ( p(x=1|y=1) * p(y=1) ) / p(x=1)')
print(f'{py_x1[1]} = ( {px_y1[1]} * {py[1]} ) / {px[1]}')
print()

print('p(y=0|x=2) = ( p(x=2|y=0) * p(y=0) ) / p(x=2)')
print(f'{py_x2[0]} = ( {px_y0[2]} * {py[0]} ) / {px[2]}')
print()

print('p(y=1|x=2) = ( p(x=2|y=1) * p(y=1) ) / p(x=2)')
print(f'{py_x2[1]} = ( {px_y1[2]} * {py[1]} ) / {px[2]}')
print()

p(y=0|x=1) = ( p(x=1|y=0) * p(y=0) ) / p(x=1)
0.0 = ( 0.0 * 0.5 ) / 0.3333333333333333

p(y=1|x=1) = ( p(x=1|y=1) * p(y=1) ) / p(x=1)
1.0 = ( 0.6666666666666666 * 0.5 ) / 0.3333333333333333

p(y=0|x=2) = ( p(x=2|y=0) * p(y=0) ) / p(x=2)
0.75 = ( 1.0 * 0.5 ) / 0.6666666666666666

p(y=1|x=2) = ( p(x=2|y=1) * p(y=1) ) / p(x=2)
0.25 = ( 0.3333333333333333 * 0.5 ) / 0.6666666666666666



### Example: Bernoulli Distributions

As an example of a specific class of named probability distributions, we briefly consider the simple *Bernoulli Distribution*. A Bernoulli distribution is the unique distribution defined on binary random variables $y=0,1$. They are completely determined by specifying a single parameter $p$ defined by $p \equiv p(y=1)$, since $p(y=0) = 1-p(y=1) = 1-p$. Bernoulli variables have the property that their mean value converges to $\mu = p$ for large samples, and their standard deviation converges to $\sigma = \sqrt{p(1-p)}$.

Bernoulli distributions are useful for modeling many binary classification problems using a probability model. Given input features $x$, we seek to estimate the prediction probabilities $p(y|x)$ from the data. Since $y$ is a binary variable, it *must* be the case that it's sampled from a Bernoulli distribution with some parameter $p$, where $p$ must be estimated from the data somehow. You might ask, why not just estimate $p$ using the mean of $y$ values, since with large enough samples? Why do we go through the effort to build all these fancy ML models instead? Think about this question for a bit and see if you can answer it...

In [108]:
p = 0.25
y = np.random.binomial(n=1, p=p, size=100)
print(f'first 10: {y[:10]}')
print(f'true mean: {p} , estimated mean: {np.mean(y)}')
print(f'true std: {np.sqrt(p*(1-p))} , estimated std: {np.std(y)}')

first 10: [1 0 1 0 0 0 0 0 0 1]
true mean: 0.25 , estimated mean: 0.29
true std: 0.4330127018922193 , estimated std: 0.45376205218153715


### Brief Aside: Continuous Distributions

I want to make a brief mention about something I've kind of glossed over. Namely, how do you define a distribution for variables that take on a *continuous* range of values?

A well-known example of a continuous distribution is the *Gaussian* or *Normal* Distribution. This is the famous "bell curve", and is used to model a host of things (e.g. measurement errors). A standard Gaussian distribution is defined on all real values, i.e. $-\infty \leq x \leq \infty$, and is defined by

$$p(x) \equiv \frac{1}{\sqrt{2\pi}}e^{-\frac{x}{2}}, \quad -\infty \leq x \leq \infty.$$

An even simpler example of a continuous distribution of interest is the standard *Uniform Distribution*, defined by 

$$p(x) \equiv 1 \text{ if } 0 \leq x \leq 1 \text{ else } 0.$$ 

This is the distribution that the *rand* function samples from, e.g. `np.random.rand()`, and is perhaps the most widely used distribution out there. But there's a problem here. We can't just define a probability for *every* $0 \leq x \leq 1$! It would be impossible. Not to mention, such probabilities couldn't even sum to one since there are an uncountably infinite number of values in the interval $0 \leq x \leq 1$.

The way around this is to use calculus. Since calculus isn't a prerequisite for this tutorial, I'll only mention this in passing for those who are familiar with integrals. The way to define probabilities on continuous variables is to define them on intervals instead of values. On values, we instead define a *probability density function* $p(x)$, and integrate over the density to get the probability of a value occuring inside the interval of interest. If one wishes to know the probability $x$ is between two values $a$ and $b$, that would be defined by specifying $p(x)$ and then integrating as follows:

$$\text{Pr}(a \leq x \leq b) \equiv \int_a^b p(x) dx.$$

Instead of requiring that probabilities be positive and sum to one for continuous variables, we require that the densities be positive and integrate to one over the whole range of values. Note as a consequence of this, densities can take on *any* positive value, and hence don't have to be less than one in general. It's the probabilities that must be between zero and one, not the densities.

The same rules mentioned before about joint distributions, conditional probabilities, and Bayes Rule all extend to continuous variables by swapping probabilities with densities.

## Naive Bayes From Scratch

### Setup

Naive Bayes is a classification algorithm, which is a supervised learning algorithm. This assumes we have labeled training data, and those labels are discrete values. Recall supervised learning assumes we have a set of labeled data $D=\{(x_1,y_1),\cdots,(x_n,y_n)\}$. As usual, the goal is to learn a prediction function $f(x)$ on the training inputs $x$ such that the training outputs $y$ match the predicted outputs given by $f(x)$. That is, we seek to find a function $f(x)$ such that $y_i \approx f(x_i)$ for each $(x_i,y_i)$ in $D$.

Focusing on text in particular, we shall assume the initial input to the algorithm is a bag-of-words vector. That is, each document of text is represented as a vector of word counts (see text classification tutorial on this). Given a vocabulary size $m$, each document vector $x$ is defined by $x = (w_1,w_2,\cdots,w_m)$, where $w_j$ is the number of times the j<sup>th</sup> word $w_j$ in the vocabulary appears in that document.

As discussed in the tex classification tutorial, most such $w_j$ will be zero, as most words in a vocabulary don't occur in any one piece of text, which means each $x$ will be a sparse vector.

In the specific example we focus on the binary classification of spam detection, hence each label $y$ is a binary value where `0=nonspam` and `1=spam`. Naive Bayes extends to multiclass problems fairly easily, but it's easiest if we focus on deriving the binary classification case first.

For completeness, an example of how to compute a bag-of-words array from an array of text is shown below. The computed values of `X` and `y` will also be used below to illustrate how Naive Bayes works.

In [142]:
y = np.array([0, 1, 1, 0, 1])
text = [
    'I hate you',
    'I love you',
    'I like you',
    'I hate hate hate you',
    'I like you I love you'
]

all_words = sorted(set(' '.join(text).split(' ')))
vocab = {word:idx for word, idx in zip(all_words, range(len(all_words)))}

X = []
for doc in text:
    ctr = Counter(doc.split(' '))
    x = [ctr[word] for word in vocab]
    X.append(x)
X = np.array(X)

print(f'columns: {list(vocab.keys())}')
for i in range(len(text)):
    print(f'{text[i]} -> {X[i]}')

columns: ['I', 'hate', 'like', 'love', 'you']
I hate you -> [1 1 0 0 1]
I love you -> [1 0 0 1 1]
I like you -> [1 0 1 0 1]
I hate hate hate you -> [1 3 0 0 1]
I like you I love you -> [2 0 1 1 2]


### Naive Bayes Derivation

Recall from the probability section that a function $y \approx f(x)$ has a probabilistic analogue $p(y|x)$. Thus, if we want to learn an ML model that allows for noise, it would be reasonable to learn the posterior probability $p(y|x)$ instead of a "point estimator" function like $f(x)$. We can recover $f(x)$ from $p(y|x)$ if we wish by just rounding the probability. 

Suppose for a second that we have a sample of text $x = (w_1,w_2,\cdots,w_{m(x)})$, where $m(x)$ in this case represents the number of words in *that* document (not the whole vocabulary, which we'll get back to in a second), and $y=0,1$. Then the goal is to "learn" the probabilities

$$p(y=0|x) = p(y=0|w_1,w_2,\cdots,w_{m(x)})$$
$$p(y=1|x) = p(y=1|w_1,w_2,\cdots,w_{m(x)})$$

for each $x,y$ pair in the dataset. But how do we do this? Recall from Bayes Rule we can express $p(y|x)$ in terms of a likelihood $p(x|y)$ and a prior $p(y)$. So

$$p(y=0|x) = \frac{p(x|y=0)p(y=0)}{p(x)} = \frac{p(w_1,w_2,\cdots,w_{m(x)}|y=0)p(y=0)}{p(x)}$$
$$p(y=1|x) = \frac{p(x|y=1)p(y=1)}{p(x)} = \frac{p(w_1,w_2,\cdots,w_{m(x)}|y=1)p(y=1)}{p(x)}.$$

What I just wrote down is true for any binary classification problem. There's nothing "naive" about it. Now we make the important assumption of Naive Bayes. 

**Naive Bayes Assumption:** The features of $x$ are independent of each other, given $y$. That is, each $w_i|y$ and $w_j|y$ are independent for all $i,j$.

This implies then that $p(w_i,w_j|y)=p(w_i|y)p(w_j|y)$ for each features $w_i$ and $w_j$ of $x$. As this is true for any pair of features, it follows that it's true for all features, and so the likelihood $p(x|y)$ factors:

$$p(x|y) = p(w_1,w_2,\cdots,w_{m(x)}|y) = \prod_{j=1}^{m(x)} p(w_j|y) = p(w_1|y)p(w_2|y)\cdots p(w_{m(x)}|y).$$

Under these assumptions, we can re-express Bayes Rule for binary classification by

$$p(y=0|x) = \frac{p(y=0)}{p(x)}\prod_{j=1}^{m(x)} p(w_j|y=0)$$
$$p(y=1|x) = \frac{p(y=1)}{p(x)}\prod_{j=1}^{m(x)} p(w_j|y=1).$$

### Prediction Rule

The goal now will be to take the Naive Bayes setup and derive a prediction rule. For the sake of shorthand, define the probability ratio of $w_j$ by

$$r_j \equiv \log \bigg (\frac{p(w_j|y=1)}{p(w_j|y=0)} \bigg ).$$

Notice that $r_j$ is equivalent to saying something about whether $w_j$ is most likely to be a positive feature or a negative feature. If $r_j > 0$, $w_j$ tends to occur more often in positive examples. If $r_j < 0$, $w_j$ tends to occur more often in negative examples. If $r_j = 0$ exactly, $w_j$ may occur equally likely in both positive and negative examples.

Now, let $[x > 0]$ represent the binarized feature vector of $x$, i.e. for $j=1,\cdots,m$:

$$[x_j > 0] \equiv (1 \text{ if } x_j > 0  \text{ else } 0 \big ) = \big (1 \text{ if } w_j \text{ in document else } 0 \big )$$ 

You can think of $[x > 0]$ as the bag-of-words $x$ with non-zero counts set to one (that's what binarizing does). Next, define the log posterior probability ratio of $x$ by

$$r(x) \equiv \log \bigg ( \frac{p(y=1|x)}{p(y=0|x)} \bigg ) 
       = \log \bigg ( \frac{p(y=1)}{p(y=0)} \prod_{j=1}^m r_j[x_j > 0] \bigg )
       = \log \bigg (\frac{p(y=1)}{p(y=0)} \bigg ) + \sum_{j=1}^m r_j[x_j > 0].$$

Since as a binary value $[x_j]$ just picks out $r_j$, which is the probability ratio of $w_j$, we can see that the following statements must be equivalent:
- given $x$, $y=1$ is more likely to occur than $y=0$,
- $p(y=1|x) > p(y=0|x)$,
- $r(x) > 0$.

Similarly, the following are equivalent statements as well:
- given $x$, $y=0$ is more likely to occur than $y=1$,
- $p(y=0|x) > p(y=1|x)$,
- $r(x) < 0$. 

Similarly, the following are equivalent:
- given $x$, $y=0$ is equally as likely to occur as $y=1$,
- $p(y=0|x) = p(y=1|x)$
- $r(x) = 0$. 

Thus, we've derived a classification rule. If for a given $x$ we have $r(x) > 0$, predict $\hat y=1$. If instead we have $r(x) < 0$, predict $\hat y=0$. 

What if $r(x)=0$? This is unlikely to happen exactly in real data generally, so to keep things simple we'll round down and assume $\hat y=0$ in that edge case.

**Classification Rule:** Given $x=(w_1,\cdots,w_m)$, calculate $r(x)$. Predict $\hat y = 1 \text{ if } r(x) > 0 \text{ else } 0$.

Aside: What's with the logs in the probability ratio definition? Do we actually need them here? In theory, no we don't need to take logs. We can just compute the ratio and give a prediction using 1 as the threshold instead of 0. The reason we take logs is because of numerical underflow. Taking the product of a lot of small floating point numbers on a computer is bad, and prone to result in the product being rounded down to 0. Logs convert products to sums, which makes this numerical underflow less likely to happen. This means results from the log ratio are more stable and less error prone than without the logs.

### Calculating the Probabilities

Until now we've dodged the question of how one would actually calculate each of these probabilities. If we can't calculate each $p(y)$ and $p(w_j|y)$ we can't calculate $r$, so we're dead in the water. Naive Bayes keeps things naive though. We're not actually going to "learn" what these probabilities are, we're just going to estimate them using frequencies calculated from the training set.

The simplest ones to estimate are $p(y=1)$ and $p(y=0)$. Suppose the training dataset contains $n$ samples, $n_1$ of which are positive samples, and $n_0$ of which are negative samples. Then

$$p(y=1) \approx \frac{n_1}{n},$$

$$p(y=0) \approx \frac{n_0}{n},$$

Now, define $b$ to be the log-ratio of priors

$$b \equiv \log \bigg ( \frac{p(y=1)}{p(y=0)} \bigg ) \approx \log \bigg ( \frac{n_1}{n_0} \bigg ).$$

This term is called the *bias*, and reflects the class imbalance in the training set. The bias is zero when the training set is balanced, negative when there is a smaller fraction of negative samples in the dataset than positive samples, and positive in the (unusual) inverse scenario.

We can of course get the probabilities $p(x|y)$ by calculating each of the probabilities $p(w_j|y)$ and multiplying them together. To calculate each $p(w_j|y)$ we can do something like the following:

$$p(w_j|y=1) \approx \frac{n_{j1}+1}{n_1+1} = \frac{\text{# times word } w_j \text{ occurs in positive samples}}{\text{# of positive samples }},$$

$$p(w_j|y=0) \approx \frac{n_{j0}+1}{n_0+1} = \frac{\text{# times word } w_j \text{ occurs in negative samples}}{\text{# of negative samples }},$$

Notice something: I added one to the numerator and denominator for each one. This seems like an odd thing to do. Shouldn't we just take the count occurance ratio for each one? What I did was an example of *Laplace Smoothing*. The idea is to pretend we saw a dummy example $x_0$ that contains each word in the training set. This makes probability estimates more stable, as it ensures that we never have $p(w_j|y) \neq 0$, which can be bad numerically. We don't want to divide by zero, or have zeros in the logs because they'll blow up. Other than this smoothing trick, we're just estimating frequencies as usual.

This means we can then calculate each of the word log probability ratios by 

$$r_j = \frac{p(w_j|y=1)}{p(w_j|y=0)} \approx \frac{\frac{n_{j1}+1}{n_1+1}}{\frac{n_{j0}+1}{n_0+1}}.$$
     
Letting $R=(r_1,\cdots,r_m)$ and $x=(x_1,\cdots,x_m)$, and using the definition of the dot product, we finally arrive at the very simple linear algebra formula for the log-ratio $r(x)$, and hence our prediction rule:

$$r(x) = \log \bigg (\frac{p(y=1)}{p(y=0)} \bigg ) + \sum_{j=1}^m [x_j > 0] r_j = b + R^T [x > 0].$$

### Multinomial Naive Bayes Algorithm

Given a training set with bag-of-words inputs $X$ of shape $(n,m)$ and corresponding labels $y$ of shape $(n,1)$:

- Calculate the label counts $n_0$ and $n_1$
- Calculate the occurance counts $n_{j1}$ and $n_{j0}$ for each feature $w_j$ with $j=1,\cdots,m$
- Calculate $b=\log \frac{n_1}{n_0}$ and $r_j=\frac{(n_{j1}+1)\div(n_1+1)}{(n_{j0}+1)\div(n_0+1)}$ for $j=1,\cdots,m$
- For each sample $x$ in $X$:
    - Calculate $r(x)=b + \sum_j r_j [x_j > 0]$
    - Classify $x$ by: $\hat y = 1 \text{ if } r(x) > 0 \text{ else } 0$
    
To do classification on new data (e.g. a test set), you just perform the last step (the loop body) over the new data. Notice this means you must always use $b$ and each $r_j$ from the *training set*. These are "learned" parameters that you're not allowed to tweak once the model is "trained".

In [150]:
n, m = X.shape

n1 = len(X[y == 1])
n0 = len(X[y == 0])

b = np.log(n1 / n0)

print(f'n1 = {n1}')
print(f'n0 = {n0}')
print(f'b = {b}')

n1 = 3
n0 = 2
b = 0.4054651081081644


In [149]:
nj1 = X[y == 1].sum(axis=0)
nj0 = X[y == 0].sum(axis=0)

R = np.log(((nj1 + 1)/(n1 + 1)) / ((nj0 + 1)/(n0 + 1)))

print(f'nj1 = {nj1}')
print(f'nj0 = {nj0}')
print(f'R = {R}')

nj1 = [4 0 2 2 4]
nj0 = [2 4 0 0 2]
R = [ 0.22314355 -1.89711998  0.81093022  0.81093022  0.22314355]


In [152]:
r = b + np.sign(X) @ R
yhat = (r > 0).astype(int)

print(f'r = {r}')
print(f'yhat = {yhat}')
print(f'y = {y}')

r = [-1.04536777  1.66268243  1.66268243 -1.04536777  2.47361264]
yhat = [0 1 1 0 1]
y = [0 1 1 0 1]


In [147]:
print(f'train accuracy: {(y == yhat).sum() / len(y)}')

train accuracy: 1.0


In [188]:
class MultinomialNaiveBayes:
    def __init__(self):
        pass
    
    def fit(self, X, y):
        n_pos = len(X[y == 1])
        n_neg = len(X[y == 0])
        self.bias = np.log(n_pos / n_neg)
        n_pos_words = X[y == 1].sum(axis=0)
        n_neg_words = X[y == 0].sum(axis=0)
        self.word_log_ratios = np.log(((n_pos_words + 1)/(n_pos + 1)) / ((n_neg_words + 1)/(n_neg + 1)))
        
    def predict(self, X):
        r = self.bias + np.sign(X) @ self.word_log_ratios.T
        yhat = (r > 0).astype(int)
        return yhat.flatten()
    
    def score(self, X, y):
        y_pred = self.predict(X)
        
        return (y == y_pred).sum() / len(y)

## Bernoulli Naive Bayes

fraction of docs that contain $w_j$, works better on short docs but fails often on longer docs

In [190]:
class BernoulliNaiveBayes:
    def __init__(self):
        pass
    
    def fit(self, X, y):
        n_pos = len(X[y == 1])
        n_neg = len(X[y == 0])
        self.bias = np.log(n_pos / n_neg)
        n_pos_words = X[y == 1].sum(axis=0)
        n_neg_words = X[y == 0].sum(axis=0)
        self.word_log_ratios = np.log(((n_pos_words + 1)/(n_pos + 1)) / ((n_neg_words + 1)/(n_neg + 1)))
        
    def predict(self, X):
        r = self.bias + X @ self.word_log_ratios.T
        yhat = (r > 0).astype(int)
        return yhat.flatten()
    
    def score(self, X, y):
        y_pred = self.predict(X)
        
        return (y == y_pred).sum() / len(y)

## Example: Spam Classification

We will now implement Naive Bayes and use it to do spam classification with the same SMS Spam Dataset from the Text Classification tutorial. Since this tutorial isn't about text, we will just copy and use the same functions from that tutorial to load and process the data into a numerical format. Our goal here is to focus on the classification piece, not the preprocessing. See the other tutorial if you're curious on the preprocessing details.

In [155]:
df = get_spam_data(balanced=True)
df.head(10)

,labels,text
0,1,someon know ask date servic contact cant guess...
1,1,updat _ xmas offer latest motorola sonyericsso...
2,0,sir wait call free pleas call
3,1,0a network allow compani bill sms respons supp...
4,1,updat _ 12mths half price orang line rental 40...
5,0,case guess see campus lodg
6,1,urgent mobil award 1 500 bonus caller prize 27...
7,0,put sign choos number pin show right ?
8,1,loan purpos 500 75 000 homeown tenant welcom p...
9,1,sunshin quiz win super soni dvd record cannam ...


In [156]:
df.labels.value_counts()

1    747
0    747
Name: labels, dtype: int64

In [162]:
text = df['text'].values
y = df['labels'].values

In [167]:
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.2)

cv = CountVectorizer(analyzer=str.split, ngram_range=(1,1), min_df=2)
X_train = cv.fit_transform(text_train).todense()
X_test = cv.transform(text_test).todense()

In [168]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1195, 1585), (1195,), (299, 1585), (299,))

In [189]:
model = MultinomialNaiveBayes()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
model.score(X_test, y_test)

0.8996655518394648

In [191]:
model = BernoulliNaiveBayes()
model.fit(np.sign(X_train), y_train)

y_pred = model.predict(np.sign(X_test))
model.score(np.sign(X_test), y_test)

0.8929765886287625

In [192]:
tfidf = TfidfVectorizer(analyzer=str.split, ngram_range=(1,1), min_df=2)
X_train = tfidf.fit_transform(text_train).todense()
X_test = tfidf.transform(text_test).todense()

model = MultinomialNaiveBayes()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
model.score(X_test, y_test)

0.919732441471572

## Multiclass Classification with Naive Bayes

This tutorial has mainly focused on binary classification, and the rule derived for Naive Bayes is specific to that case. Extending to the multiclass situation is actually not that difficult. We just need to tweak a couple of definitions.

Suppose we have again a labeled dataset $D$, but this time each label $y$ can take on one of $K$ classes $c_1,\cdots,c_K$.

For each class $c_k$ and each input vector $x$, Bayes Rule says

$$p(y=c_k|x) = \frac{p(x|y=c_k)p(y=c_k)}{p(x)}.$$

In the Naive Bayes setting we still assume features are conditionally independent given $y$. Thus,

$$p(y=c_k|x) = \frac{p(y=c_k)}{p(x)} \prod_j p(w_j|y=c_k).$$

In the multiclass setting, the log probability ratios need to shift to a "one vs all" format. This essentially means a "negative example" becomes "any example that has a class different from $c_k$". This means each feature $w_j$ will now have $K$ separate log probability ratios $r_{jk}$ for $k=1,\cdots,K$ with each given by 

$$r_{jk} \equiv \frac{p(w_j|y=c_k)}{p(w_j|y\neq c_k)}.$$

Then, $r_j$ from before becomes a vector of length $K$, so $r_j = (r_{j1}, \cdots, r_{jK})$, and $R=(r_1,\cdots,r_m)$ becomes a matrix of shape $(K,m)$.

The bias values must change to a one vs all format as well. For $c_k$, define the bias $b_k$ by

$$b_k \equiv \log \bigg ( \frac{p(y=c_k)}{p(y\neq c_k)} \bigg ) \approx \log \bigg ( \frac{n_k}{n-n_k} \bigg ).$$

This again means the bias $b$ will become a vector of length $K$, with $b=(b_1,\cdots,b_K)$.

We have everything we need, then, to define the log probability ratio $r(x)$, along with the classification rule. As there are now $K$ ratios, $r(x)$ must be a vector of length $K$, so $r(x)=(r_1(x),\cdots,r_K(x))$. As there are now $K$ different log-ratios to pick from, we simply select the one with the maximum value to be the prediction $\hat y$:

$$\hat y = \text{argmax}_{k=1,\cdots,K}r(x).$$

Stare at Bayes Rule for a second and you'll notice that it also must be true that

$$\hat y = \text{argmax}_{k} p(y=c_k|x) = \text{argmax}_{k} \bigg (p(y=c_k)p(x|y=c_k) \bigg ),$$

which hopefully makes some intuitive sense. You're selecting the most likely class $c_k$ to fit $x$.